In [30]:
import numpy as np
import pandas as pd
import pickle
pd.options.display.max_columns=None
import xgboost as xgb

In [31]:
test_set_1 = pd.read_csv('C:/Users/mvkri/OneDrive/Documents/DMT/modified_test_data_prob.csv',nrows=249976)
filename=('C:/Users/mvkri/OneDrive/Documents/DMT/Xgboost_model_learning.sav')
loaded_model = pickle.load(open(filename, 'rb'))

In [32]:
len(test_set_1)

249976

In [33]:
test_set_1 = test_set_1.loc[:,~test_set_1.columns.str.contains('^Unnamed')]

In [34]:
df_test = test_set_1.copy()

In [35]:
df_test = df_test.loc[:,~df_test.columns.str.contains('prop_id')]

In [36]:
df_test.head()

,srch_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate_percent_diff,comp2_rate_percent_diff,comp3_rate_percent_diff,comp4_rate_percent_diff,comp5_rate_percent_diff,comp6_rate_percent_diff,comp7_rate_percent_diff,comp8_rate_percent_diff,starrating_diff,usd_diff,prop_starating_monotonic,count_hotel,prob_book,prob_click,visitor_hist_starrating_median,visitor_hist_starrating_mean,visitor_hist_starrating_std,visitor_hist_adr_usd_median,visitor_hist_adr_usd_mean,visitor_hist_adr_usd_std,prop_starrating_median,prop_starrating_mean,prop_starrating_std,prop_review_score_median,prop_review_score_mean,prop_review_score_std,prop_brand_bool_median,prop_brand_bool_mean,prop_brand_bool_std,prop_location_score1_median,prop_location_score1_mean,prop_location_score1_std,prop_location_score2_median,prop_location_score2_mean,prop_location_score2_std,prop_log_historical_price_median,prop_log_historical_price_mean,prop_log_historical_price_std,price_usd_median,price_usd_mean,price_usd_std,promotion_flag_median,promotion_flag_mean,promotion_flag_std,srch_length_of_stay_median,srch_length_of_stay_mean,srch_length_of_stay_std,srch_booking_window_median,srch_booking_window_mean,srch_booking_window_std,srch_adults_count_median,srch_adults_count_mean,srch_adults_count_std,srch_children_count_median,srch_children_count_mean,srch_children_count_std,srch_room_count_median,srch_room_count_mean,srch_room_count_std,srch_saturday_night_bool_median,srch_saturday_night_bool_mean,srch_saturday_night_bool_std,srch_query_affinity_score_median,srch_query_affinity_score_mean,srch_query_affinity_score_std,orig_destination_distance_median,orig_destination_distance_mean,orig_destination_distance_std,random_bool_median,random_bool_mean,random_bool_std,comp1_rate_percent_diff_median,comp1_rate_percent_diff_mean,comp1_rate_percent_diff_std,comp2_rate_percent_diff_median,comp2_rate_percent_diff_mean,comp2_rate_percent_diff_std,comp3_rate_percent_diff_median,comp3_rate_percent_diff_mean,comp3_rate_percent_diff_std,comp4_rate_percent_diff_median,comp4_rate_percent_diff_mean,comp4_rate_percent_diff_std,comp5_rate_percent_diff_median,comp5_rate_percent_diff_mean,comp5_rate_percent_diff_std,comp6_rate_percent_diff_median,comp6_rate_percent_diff_mean,comp6_rate_percent_diff_std,comp7_rate_percent_diff_median,comp7_rate_percent_diff_mean,comp7_rate_percent_diff_std,comp8_rate_percent_diff_median,comp8_rate_percent_diff_mean,comp8_rate_percent_diff_std
0,1,216,-10.0,-10.0,3,4.5,1,2.94,0.0691,5.03,119.0,0,1,10,2,0,1,0,-10.0,-10.0,0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,13.0,129.0,0.31206,15,0.466667,0.466667,-10.0,-9.406550,2.714516,-10.0,-2.765876,38.350735,3,3,0.0,4.5,4.5,0.0,1,1,0.0,2.94,2.94,0.0,0.0251,-0.768651,2.741913,5.02,4.196765,1.864181,125.0,125.329946,12.255565,0.0,0.056604,0.231396,1.0,1.528302,1.016689,8.0,22.824798,37.344764,2.0,1.924528,0.903278,0.0,0.336927,0.776138,1.0,1.083558,0.404066,1.0,0.568733,0.495922,-10.0,-10.951761,4.808232,172.67,711.473827,930.967214,0.0,0.183288,0.387425,-10.0,-10.0,0.0,-10.0,-5.700809,10.657289,-10.0,-6.991914,11.532911,-10.0,-10.0,0.0,-10.0,-10.000000,0.000000,-10.0,-10.0,0.0,-10.0,-10.0,0.0,-10.0,-7.188679,6.939272
1,1,216,-10.0,-10.0,3,4.5,1,2.64,0.0843,4.93,118.0,0,1,10,2,0,1,0,-10.0,-10.0,0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,13.0,128.0,0.31206,32,0.406250,0.468750,-10.0,-9.523044,2.463008,-10.0,-3.844012,35.939700,3,3,0.0,4.5,4.5,0.0,1,1,0.0,2.64,2.64,0.0,0.0106,-0.690039,2.607302,4.93,4.283044,1.679731,116.0,116.769375,20.625295,0.0,0.018145,0.133611,1.0,1.625000,1.077080,8.0,21.245968,35.514458,2.0,2.024194,0.864227,0.0,0.445565,0.836697,1.0,1.0725

In [39]:
def predict(model, df):
    return model.predict(df.loc[:, ~df.columns.isin(['srch_id'])])

In [40]:
unique_groups=df_test['srch_id'].unique().tolist()
#unique_groups=[1]
prop_id_list_test = []
# for rows in unique_groups:
#     df = df_test.loc[df_test.srch_id == rows]
#     prop_id_list_test.append(df['prop_id'].tolist())
# flat_list = []
# for sublist in prop_id_list_test:
#     for item in sublist:
#         flat_list.append(item)
search_id_test = []
ordering_test = []
for rows in unique_groups:
    df = df_test.loc[df_test.srch_id == rows]
    #df['id'] = df['srch_id']
    predictions_test = (df.groupby('srch_id')
               .apply(lambda x: predict(loaded_model, x)))
    List_value_test = list(predictions_test)
    prop_id_list_test.append(test_set_1.loc[test_set_1['srch_id'] == rows, 'prop_id'].tolist())
    i = 0;
    for values in List_value_test[i]:
        #print('inside loop')
        id_value = int(rows)
        #X_final_res_test = X_final_res_test.append(pd.Series({'srch_id': id_value, 'ordering': values}),ignore_index=True)
        search_id_test.append(id_value)
        ordering_test.append(values)
       
    #X_eval.loc[X_eval.srch_id == row.srch_id, 'Ordering'] = pd.Series(List_value[i])
    #X_eval.loc[X_eval['srch_id'] == row.srch_id] = List_value[i]
    i += 1
flat_list = []
for sublist in prop_id_list_test:
    for item in sublist:
        flat_list.append(item)
required_columns = ['srch_id', 'ordering']
X_final_res_test = pd.DataFrame(columns=required_columns)
X_final_res_test["srch_id"] = search_id_test
X_final_res_test["srch_id"] = pd.to_numeric(X_final_res_test["srch_id"],downcast='integer')
X_final_res_test["ordering"] = ordering_test
X_final_res_test['prop_id'] = flat_list

In [41]:
X_final_res_test.to_csv('C:/Users/mvkri/OneDrive/Documents/DMT/Lambda_1_result.csv')

In [42]:
#del test_set_1
del df_test
del X_final_res_test

In [43]:
test_set_2 = pd.read_csv('C:/Users/mvkri/OneDrive/Documents/DMT/modified_test_data.csv',skiprows=249976,header=None,names=test_set_1.columns)
del test_set_1

In [44]:
test_set_2 = test_set_2.loc[:,~test_set_2.columns.str.contains('^Unnamed')]

In [45]:
df_test = test_set_2.copy()

In [46]:
df_test = df_test.loc[:,~df_test.columns.str.contains('prop_id')]

In [ ]:
unique_groups=df_test['srch_id'].unique().tolist()
#unique_groups=[1]
prop_id_list_test = []
# for rows in unique_groups:
#     df = df_test.loc[df_test.srch_id == rows]
#     prop_id_list_test.append(df['prop_id'].tolist())
# flat_list = []
# for sublist in prop_id_list_test:
#     for item in sublist:
#         flat_list.append(item)
search_id_test = []
ordering_test = []
for rows in unique_groups:
    df = df_test.loc[df_test.srch_id == rows]
    #df['id'] = df['srch_id']
    predictions_test = (df.groupby('srch_id')
               .apply(lambda x: predict(loaded_model, x)))
    List_value_test = list(predictions_test)
    prop_id_list_test.append(test_set_2.loc[test_set_2['srch_id'] == rows, 'prop_id'].tolist())
    i = 0;
    for values in List_value_test[i]:
        #print('inside loop')
        id_value = int(rows)
        #X_final_res_test = X_final_res_test.append(pd.Series({'srch_id': id_value, 'ordering': values}),ignore_index=True)
        search_id_test.append(id_value)
        ordering_test.append(values)
       
    #X_eval.loc[X_eval.srch_id == row.srch_id, 'Ordering'] = pd.Series(List_value[i])
    #X_eval.loc[X_eval['srch_id'] == row.srch_id] = List_value[i]
    i += 1
flat_list = []
for sublist in prop_id_list_test:
    for item in sublist:
        flat_list.append(item)
required_columns = ['srch_id', 'ordering']
X_final_res_test = pd.DataFrame(columns=required_columns)
X_final_res_test["srch_id"] = search_id_test
X_final_res_test["srch_id"] = pd.to_numeric(X_final_res_test["srch_id"],downcast='integer')
X_final_res_test["ordering"] = ordering_test
X_final_res_test['prop_id'] = flat_list

In [ ]:
X_final_res_test.to_csv('C:/Users/mvkri/OneDrive/Documents/DMT/Lambda_2_result.csv')

In [ ]:
del test_set_2
del df_test

In [ ]:
res2 = pd.read_csv('C:/Users/mvkri/OneDrive/Documents/DMT/Lambda_1_result.csv')

In [ ]:
#res2.sort_values(['srch_id', 'ordering'], ascending = (True, False), inplace=True)

In [ ]:
len(X_final_res_test)

In [ ]:
res2 = res2.append(X_final_res_test)

In [ ]:
len(res2)

In [ ]:
res2.sort_values(['srch_id', 'ordering'], ascending = (True, False), inplace=True)

In [ ]:
len(X_final_res_test)

In [ ]:
res2 = res2.loc[:,~res2.columns.str.contains('^Unnamed')]
del res2['ordering']


In [ ]:
res2.tail()

In [ ]:
res2.to_csv('C:/Users/mvkri/OneDrive/Documents/DMT/Lambda_merged_result_3.csv',index=False)

In [ ]:
len(res2)